In [2]:
# Import necessary libraries
import os
from dotenv import load_dotenv
load_dotenv()
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.tools import ToolContext

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [3]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nOpenAI is ready!")

ModelResponse(id='chatcmpl-CACJlkrmafO8YNHdPhFkdzrE8yrlO', created=1756546665, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_80956533cb', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready. How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready!


## 4.3. Define the User Intent Agent

### 4.3.1 Agent Instructions

In [5]:
# define the role and goal for the user intent agent
agent_role_and_goal = """
    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.
"""

In [4]:
# give the agent some hints about what to say
agent_conversational_hints = """
    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - risk analysis for big companies, maybe scrapping news and so on
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music
"""

In [6]:
# describe what the output should look like
agent_output_definition = """
    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alternatives."
"""

In [7]:
# specify the steps the agent should follow
agent_chain_of_thought_directions = """
    Think carefully and collaborate with the user:
    1. Understand the user's goal, which is a kind_of_graph with description
    2. Ask clarifying questions as needed
    3. When you think you understand their goal, use the 'set_perceived_user_goal' tool to record your perception
    4. Present the perceived user goal to the user for confirmation
    5. If the user agrees, use the 'approve_perceived_user_goal' tool to approve the user goal. This will save the goal in state under the 'approved_user_goal' key.
"""

In [8]:
# combine all the instruction components into one complete instruction...
complete_agent_instruction = f"""
{agent_role_and_goal}
{agent_conversational_hints}
{agent_output_definition}
{agent_chain_of_thought_directions}
"""

print(complete_agent_instruction)




    You are an expert at knowledge graph use cases. 
    Your primary goal is to help the user come up with a knowledge graph use case.


    If the user is unsure what to do, make some suggestions based on classic use cases like:
    - risk analysis for big companies, maybe scrapping news and so on
    - social network involving friends, family, or professional relationships
    - logistics network with suppliers, customers, and partners
    - recommendation system with customers, products, and purchase patterns
    - fraud detection over multiple accounts with suspicious patterns of transactions
    - pop-culture graphs with movies, books, or music


    A user goal has two components:
    - kind_of_graph: at most 3 words describing the graph, for example "social network" or "USA freight logistics"
    - description: a few sentences about the intention of the graph, for example "A dynamic routing and delivery system for cargo." or "Analysis of product dependencies and supplier alte

### Tool Definition

In [9]:
# Tool: Set Perceived User Goal
# to encourage collaboration with the user, the first tool only sets the perceived user goal

PERCEIVED_USER_GOAL = "perceived_user_goal"

def set_perceived_user_goal(kind_of_graph: str, graph_description:str, tool_context: ToolContext):
    """Sets the perceived user's goal, including the kind of graph and its description.
    
    Args:
        kind_of_graph: 2-3 word definition of the kind of graph, for example "recent US patents"
        graph_description: a single paragraph description of the graph, summarizing the user's intent
    """
    user_goal_data = {"kind_of_graph": kind_of_graph, "graph_description": graph_description}
    tool_context.state[PERCEIVED_USER_GOAL] = user_goal_data
    return tool_success(PERCEIVED_USER_GOAL, user_goal_data)


In [10]:
# Tool: Approve the perceived user goal
# approval from the user should trigger a call to this tool

APPROVED_USER_GOAL = "approved_user_goal"

def approve_perceived_user_goal(noop: str,tool_context: ToolContext):
    """Upon approval from user, will record the perceived user goal as the approved user goal.
    
    Only call this tool if the user has explicitly approved the perceived user goal.
    Args:
        noop: no op parameter
    """
    # Trust, but verify. 
    # Require that the perceived goal was set before approving it. 
    # Notice the tool error helps the agent take
    if PERCEIVED_USER_GOAL not in tool_context.state:
        return tool_error("perceived_user_goal not set. Set perceived user goal first, or ask clarifying questions if you are unsure.")
    
    tool_context.state[APPROVED_USER_GOAL] = tool_context.state[PERCEIVED_USER_GOAL]

    return tool_success(APPROVED_USER_GOAL, tool_context.state[APPROVED_USER_GOAL])


In [11]:
# add the tools to a list
user_intent_agent_tools = [set_perceived_user_goal, approve_perceived_user_goal]

### 4.3.3 Agent Definition

In [13]:
user_intent_agent = Agent(
    name="user_intent_agent_v1",
    model=llm,
    description="Helps the user ideate on a knowledge graph use case.",
    instruction=complete_agent_instruction,
    tools=[set_perceived_user_goal, approve_perceived_user_goal]
)
print(f"Agent '{user_intent_agent.name}' created.")

Agent 'user_intent_agent_v1' created.


## 4.4. Interact with the Agent

In [14]:
# use a helper to create an agent execution environment
from helper import make_agent_caller

# NOTE: if re-running the session, come back here to re-initialize the agent
user_intent_caller = await make_agent_caller(user_intent_agent)

In [15]:
# Run the Initial Conversation

session_start = await user_intent_caller.get_session()
print(f"Session Start: {session_start.state}") # expect this to be empty

# We need an async function to await for each conversation
async def run_conversation():
    # start things off by describing your goal
    await user_intent_caller.call("""I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.""") 

    if PERCEIVED_USER_GOAL not in session_start.state:
        # the LLM may have asked a clarifying question. offer some more details
        await user_intent_caller.call("""I'm concerned about possible manufacturing or supplier issues.""")        

    # Optimistically presume approval.
    await user_intent_caller.call("Approve that goal.", True)

await run_conversation()

session_end = await user_intent_caller.get_session()

Session Start: {}

>>> User Query: I'd like a bill of materials graph (BOM graph) which includes all levels from suppliers to finished product, 
    which can support root-cause analysis.
<<< Agent Response: The idea of a Bill of Materials (BOM) graph sounds like a great use case. To make sure I fully understand your intention, I'd like to ask a few clarifying questions:

1. Are you focusing on any specific industry or type of product for this BOM graph?
2. Do you want to include any specific types of root-cause analysis, such as defect tracking, supply chain disruptions, or other issues?
3. How would you like to visualize or utilize the output from the root-cause analysis? For example, are you planning to use it for real-time monitoring, reporting, or decision-making processes? 

These additional details will help me capture your goal more accurately.

>>> User Query: I'm concerned about possible manufacturing or supplier issues.
<<< Agent Response: Thank you for the clarification! He